# Getting Started with NVFlare (TensorFlow)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/NVFlare/blob/main/examples/getting_started/tf/nvflare_tf_getting_started.ipynb)

NVFlare is an open-source framework that allows researchers and
data scientists to seamlessly move their machine learning and deep
learning workflows into a federated paradigm.

## Basic Concepts
At the heart of NVFlare lies the concept of collaboration through
"tasks." An FL controller assigns tasks (e.g., training on local data) to one or more FL clients, processes returned
results (e.g., model weight updates), and may assign additional
tasks based on these results and other factors (e.g., a pre-configured
number of training rounds). The clients run executors which can listen for tasks and perform the necessary computations locally, such as model training. This task-based interaction repeats
until the experiment’s objectives are met. 

<img src="../../../docs/resources/controller_executor_no_filter.png" alt="NVIDIA FLARE Controller and Executor" width=75% height=75% />

## Setup environment

Install nvflare and dependencies:

In [ ]:
! pip install --ignore-installed blinker
! pip install nvflare~=2.5.0rc tensorflow[and-cuda]

If running in Google Colab, download the source code for this example:

In [ ]:
! npx degit -f NVIDIA/NVFlare/examples/getting_started/tf/src src

## Federated Averaging with NVFlare
Given the flexible controller and executor concepts, it is easy to implement different computing & communication patterns with NVFlare, such as [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) and [cyclic weight transfer](https://academic.oup.com/jamia/article/25/8/945/4956468). 

The controller's `run()` routine is responsible for assigning tasks and processing task results from the Executors. 

### Server Code
First, we provide a simple implementation of the [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) algorithm with NVFlare. 
The `run()` routine implements the main algorithmic logic. 
Subroutines, like `sample_clients()` and `scatter_and_gather_model()` utilize the communicator object, native to each Controller to get the list of available clients,
distribute the current global model to the clients, and collect their results.

The FedAvg controller implements these main steps:
1. FL server initializes an initial model using `self.load_model()`.
2. For each round (global iteration):
    - FL server samples available clients using `self.sample_clients()`.
    - FL server sends the global model to clients and waits for their updates using `self.send_model_and_wait()`.
    - FL server aggregates all the `results` and produces a new global model using `self.update_model()`.

```python
class FedAvg(BaseFedAvg):
    def run(self) -> None:
        self.info("Start FedAvg.")

        model = self.load_model()
        model.start_round = self.start_round
        model.total_rounds = self.num_rounds

        for self.current_round in range(self.start_round, self.start_round + self.num_rounds):
            self.info(f"Round {self.current_round} started.")
            model.current_round = self.current_round

            clients = self.sample_clients(self.num_clients)

            results = self.send_model_and_wait(targets=clients, data=model)

            aggregate_results = self.aggregate(results)

            model = self.update_model(model, aggregate_results)

            self.save_model(model)

        self.info("Finished FedAvg.")
```

### Client Code 
Given a CIFAR10 [TensorFlow](https://www.tensorflow.org/) code example with a network defined at [src/tf_net.py](src/tf_net.py), we wish to adapt this centralized training code to something that can run in a federated setting.


On the client side, the training workflow is as follows:
1. Receive the model from the FL server.
2. Perform local training on the received global model
and/or evaluate the received global model for model
selection.
3. Send the new model back to the FL server.

Using NVFlare's client API, we can easily adapt machine learning code that was written for centralized training and apply it in a federated scenario.
For a general use case, there are three essential methods to achieve this using the Client API :
- `init()`: Initializes NVFlare Client API environment.
- `receive()`: Receives model from the FL server.
- `send()`: Sends the model to the FL server.

With these simple methods, the developers can use the Client API
to change their centralized training code to an FL scenario with
five lines of code changes as shown below.
```python
    import nvflare.client as flare
    
    flare.init() # 1. Initializes NVFlare Client API environment.
    input_model = flare.receive() # 2. Receives model from the FL server.
    for k, v in input_model.params.items():
        model.get_layer(k).set_weights(v) # 3. Loads model from NVFlare
    
    # original local training code
    model.fit(...)
    
    output_model = flare.FLModel(params={layer.name: layer.get_weights() for layer in model.layers}) # 4. Put the results in a new `FLModel`
    flare.send(output_model) # 5. Sends the model to the FL server.  
```

The full client training script is saved in a separate file, e.g. [./src/cifar10_tf_fl.py](./src/cifar10_tf_fl.py) doing CNN training on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

## Run an NVFlare Job
Now that we have defined the FedAvg controller to run our federated compute workflow on the FL server, and our client training script to receive the global models, run local training, and send the results back to the FL server, we can put everything together using NVFlare's Job API.

#### 1. Define the initial model
First, we define the global model used to initialize the model on the FL server. See [src/tf_net.py](src/tf_net.py).

```python
from tensorflow.keras import layers, models

class TFNet(models.Sequential):
    def __init__(self, input_shape=(None, 32, 32, 3)):
        super().__init__()
        self._input_shape = input_shape
        # Do not specify input as we will use delayed built only during runtime of the model
        # self.add(layers.Input(shape=(32, 32, 3)))
        self.add(layers.Conv2D(32, (3, 3), activation="relu"))
        self.add(layers.MaxPooling2D((2, 2)))
        self.add(layers.Conv2D(64, (3, 3), activation="relu"))
        self.add(layers.MaxPooling2D((2, 2)))
        self.add(layers.Conv2D(64, (3, 3), activation="relu"))
        self.add(layers.Flatten())
        self.add(layers.Dense(64, activation="relu"))
        self.add(layers.Dense(10))
```

#### 2. Define a FedJob
The `FedJob` is used to define how controllers and executors are placed within a federated job using the `to(object, target)` routine.

Here we use a TensorFlow `BaseFedJob`, where we can define the job name and the initial global model.
The `BaseFedJob` automatically configures components for model persistence, model selection, and TensorBoard streaming for convenience.

In [ ]:
from src.tf_net import TFNet

from nvflare.app_common.workflows.fedavg import FedAvg
from nvflare.app_opt.tf.job_config.base_fed_job import BaseFedJob
from nvflare.job_config.script_runner import FrameworkType, ScriptRunner

job = BaseFedJob(
    name="cifar10_tf_fedavg",
    initial_model=TFNet(),
)

#### 3. Define the Controller Workflow
Define the controller workflow and send to server.

In [ ]:
n_clients = 2

controller = FedAvg(
    num_clients=n_clients,
    num_rounds=2,
)
job.to(controller, "server")

That completes the components that need to be defined on the server.

#### 4. Add clients
Next, we can use the `ScriptRunner` and send it to each of the clients to run our training script.

Note that our script could have additional input arguments, such as batch size or data path, but we don't use them here for simplicity.

In [ ]:
for i in range(n_clients):
    runner = ScriptRunner(
        script="src/cifar10_tf_fl.py", script_args="",  # f"--batch_size 32 --data_path /tmp/data/site-{i}"
        framework=FrameworkType.TENSORFLOW,
    )
    job.to(runner, f"site-{i+1}")

That's it!

#### 5. Optionally export the job
Now, we could export the job and submit it to a real NVFlare deployment using the [Admin client](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html) or [FLARE API](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html). 

In [ ]:
job.export_job("/tmp/nvflare/jobs/job_config")

#### 6. Run FL Simulation
Finally, we can run our FedJob in simulation using NVFlare's [simulator](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/fl_simulator.html) under the hood. We can also specify which GPU should be used to run this client, which is helpful for simulated environments. The results will be saved in the specified `workdir`.

In [ ]:
job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="0")

If using Google Colab and the output is not showing correctly, export the job and run it with the simulator command instead:

In [ ]:
! nvflare simulator -w /tmp/nvflare/jobs/workdir -n 2 -t 2 -gpu 0 /tmp/nvflare/jobs/job_config/cifar10_tf_fedavg

#### 7. Next steps

Continue with the steps described in the [README.md](README.md) to run more experiments with a more complex model and more advanced FL algorithms. 